In [18]:
from collections import defaultdict
import random
import math

def ADDCOUNTS(word,topic,docid,amount):
    xcounts[topic]+=amount
    xcounts[f"{word}|{topic}"]+=amount
    ycounts[docid]+=amount
    ycounts[f"{topic}|{docid}"]+=amount

def SampleOne(probs):
    z=sum(probs)
    remaining=random.uniform(0,z)
    for i in range(len(probs)):
        remaining-=probs[i]
        if remaining<=0:
            return i
    
xcorpus=[]
ycorpus=[]
xcounts=defaultdict(lambda:0)
ycounts=defaultdict(lambda:0)
NUM_TOPICS=2
with open("wiki-en-documents.txt","r",encoding="utf-8") as f:
    for line in f:
        docid=len(xcorpus)
        words=line.strip().split(" ")
        topics=[]
        for word in words:
            topic=random.randint(0,NUM_TOPICS-1)
            topics.append(topic)
            ADDCOUNTS(word,topic,docid,1)
        xcorpus.append(words)
        ycorpus.append(topics)

iterations=10
for num in range(iterations):
    ll=0
    for i in range(len(xcorpus)):
        for j in range(len(xcorpus[i])):
            x=xcorpus[i][j]
            y=ycorpus[i][j]
            ADDCOUNTS(x,y,i,-1)
            probs=[]
            for k in range(NUM_TOPICS):
                alpha=0.01
                beta=0.01
                P_xk=(xcounts[f"{x}|{k}"]+alpha)/(xcounts[k]+alpha*len(xcorpus))
                P_ky=(ycounts[f"{k}|{y}"]+beta)/(ycounts[y]+beta*len(ycorpus))
                probs.append(P_xk*P_ky)
            new_y=SampleOne(probs)
            ll+=math.log(probs[new_y])
            ADDCOUNTS(x,new_y,i,1)
            ycorpus[i][j]=new_y

for i in range(len(xcorpus)):
    for j in range(len(xcorpus[i])):
        x=xcorpus[i][j]
        y=ycorpus[i][j]
        if y==0:
            topic0[x]+=1
        else:
            topic1[x]+=1
print("topic0")
print(sorted(topic0.items(),key=lambda x:x[1],reverse=True)[:20])
print("\ntopic1")
print(sorted(topic1.items(),key=lambda x:x[1],reverse=True)[:20])

topic0
[('the', 142678), (',', 121247), ('of', 74689), ('.', 66502), ('and', 54756), ('in', 45030), (')', 43049), ('(', 40542), ('to', 40164), ('a', 31175), ('was', 28195), ('&apos;', 20600), ('is', 18757), ('as', 17606), ('&quot;', 17313), ('that', 13666), ('it', 13323), ('on', 13065), ('for', 12187), ('by', 12159)]

topic1
[('the', 178634), (',', 144397), ('of', 89627), ('.', 82042), ('and', 66736), ('(', 54890), ('in', 52522), (')', 52287), ('to', 47812), ('a', 40969), ('was', 34617), ('&quot;', 24527), ('is', 24443), ('as', 23598), ('&apos;', 21540), ('that', 16150), ('by', 15945), ('for', 14453), ('it', 14453), ('from', 14363)]
